In [8]:
import cv2 
import numpy as np
import math
# 将图片文件命名为fy.jpg,与程序放在同一目录下
img = cv2.imread('1.jpg')
rows, cols, chnl= img.shape
x1 = np.linspace(0,90,200)
data1 = []
for jiao in x1:
    #旋转参数：旋转中心，旋转角度，scale
    M = cv2.getRotationMatrix2D((cols/2, rows/2), jiao, 1)
    #参数：原始图像，旋转参数，元素图像宽高
    rotated = cv2.warpAffine(img, M, (cols, rows))

    grayimg = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)

    # 二值化
    ret, binary_img = cv2.threshold(grayimg, 240, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    for cont in contours:
        # 对每个轮廓点求最小外接矩形
        rect = cv2.minAreaRect(cont)
        # cv2.boxPoints可以将轮廓点转换为四个角点坐标
        box = cv2.boxPoints(rect)
        # 这一步不影响后面的画图，但是可以保证四个角点坐标为顺时针
        startidx = box.sum(axis=1).argmin()
        box = np.roll(box,4-startidx,0)
        # 在原图上画出预测的外接矩形
        box = box.reshape((-1,1,2)).astype(np.int32)
        cv2.polylines(grayimg,[box],True,(0,255,0),1)
        data = []
        for i in box:
            data.append(list(i[0]))
        data.sort()
        op1=math.pow(data[0][0]-data[1][0], 2) + math.pow(data[0][1]-data[1][1], 2)
        op2=math.pow(data[0][0]-data[2][0], 2) + math.pow(data[0][1]-data[2][1], 2)
        data = []
        if op1 > 10 and op1 != 98596 and jiao != 0:
            data1.append([jiao, abs((op1/op2)-1), op1, op2])
data1.sort()
# 正方形的边长
length = float(math.sqrt(data1[2][2]))
print(f"旋转角度为{data1[0][0]}, 正方形的边长为{length}")

旋转角度为0.45226130653266333, 正方形的边长为285.72714256787015


In [15]:
# 边缘提取
# xgrd = cv2.Sobel(binary_img,cv2.CV_16SC1,1,0)
# ygrd = cv2.Sobel(binary_img,cv2.CV_16SC1,0,1)

# egde_output = cv2.Canny(xgrd,ygrd,50,150)
# cv2.imshow('canny_edge',egde_output)
# cv2.waitKey(0)
# cv2.imwrite('gray.jpg', egde_output)
# cv2.destroyAllWindows()
# plt.imshow(egde_output, cmap="gray")
# plt.savefig()
314*314

98596

In [40]:
# Canny边缘特征提取
# blurred = cv2.GaussianBlur(img,(11,11),0)
# gaussImg = cv2.Canny(blurred, 10, 70)
# plt.imshow(gaussImg, cmap='gray')
# cv2.imshow("Img",gaussImg)
# cv2.waitKey(0)
# for i in gaussImg:
#     print(i)
# print(len(gaussImg))

In [7]:
img = cv2.imread('1.jpg')
rows, cols, chnl= img.shape
#旋转参数：旋转中心，旋转角度，scale
M = cv2.getRotationMatrix2D((cols/2, rows/2), 60, 1)
#参数：原始图像，旋转参数，元素图像宽高
rotated = cv2.warpAffine(img, M, (cols, rows))
grayimg = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
ret, binary_img = cv2.threshold(grayimg, 240, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(binary_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
for cont in contours:
    # 对每个轮廓点求最小外接矩形
    rect = cv2.minAreaRect(cont)
#         cv2.boxPoints可以将轮廓点转换为四个角点坐标
    box = cv2.boxPoints(rect)
#         这一步不影响后面的画图，但是可以保证四个角点坐标为顺时针
    startidx = box.sum(axis=1).argmin()
    box = np.roll(box,4-startidx,0)
#         在原图上画出预测的外接矩形
    box = box.reshape((-1,1,2)).astype(np.int32)
    cv2.polylines(grayimg,[box],True,(0,255,0),1)
    data = []
    for i in box:
        data.append(list(i[0]))
    data.sort()
    print(box)

[[[  0   0]]

 [[341   0]]

 [[341 313]]

 [[  0 313]]]
[[[-24 113]]

 [[189 -36]]

 [[354 198]]

 [[140 348]]]


In [1]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform, measure
from scipy.spatial import ConvexHull

# 读取枫叶图像，并转换为灰度图像
img = io.imread('fy.jpg')
gray = transform.rgb2gray(img)

# 找到枫叶的轮廓
contours = measure.find_contours(gray, 0.5)
max_contour = max(contours, key=len)

# 计算枫叶的凸包
hull = ConvexHull(max_contour)
hull_points = max_contour[hull.vertices]

# 计算枫叶的最小外接矩形
rect = measure.min_area_rect(hull_points)
rect_points = np.array(rect[1])

# 计算使得长边和短边相等的旋转角度
rect_angle = rect[2]
square_angle = rect_angle - np.arctan(rect[0][1] / rect[0][0])

# 计算此时该外接正方形的边长
square_side = np.sqrt(rect[0][0] ** 2 + rect[0][1] ** 2)

# 打印结果
print(f'从枫叶当前状态旋转最小角度为{np.rad2deg(square_angle):.2f}°')
print(f'此时该外接正方形的边长为{square_side:.2f}像素')

# 绘制枫叶、凸包和最小外接矩形
plt.plot(max_contour[:, 1], max_contour[:, 0], linewidth=2, label='Leaf contour')
plt.plot(hull_points[:, 1], hull_points[:, 0], 'r--', linewidth=2, label='Convex hull')
plt.plot(rect_points[:, 0], rect_points[:, 1], 'g-', linewidth=2, label='Minimum area rectangle')
plt.axis('equal')
plt.legend()
plt.title('Leaf, convex hull and minimum area rectangle')
plt.show()

ModuleNotFoundError: No module named 'skimage'